In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

from IJCAI2017_TOOL import *

In [2]:
#引入商家类别,加入独热编码
category = pd.read_csv('./Flamingo_Rank4_code/data_new/SHOP_CAT.csv')
one_hot = pd.get_dummies(category['CAT'])
category = category.join(one_hot)
SHOP_SJ = map(lambda x:'SJ'+ str(x).zfill(2), np.arange(15)) 
category.columns = ['SHOP_CA1_EN','SHOP_CA2_EN','SHOP_CA3_EN','Num', 'CAT'] + SHOP_SJ

In [3]:
category.head()

,SHOP_CA1_EN,SHOP_CA2_EN,SHOP_CA3_EN,Num,CAT,SJ00,SJ01,SJ02,SJ03,SJ04,SJ05,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14
0,Delicious food,Baking pastry,Bread,71,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Delicious food,Baking pastry,Cake,39,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Delicious food,Baking pastry,Other baked cakes,12,Baking pastry,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Delicious food,Chinese food,Fujian Cuisine,1,Chinese food,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Delicious food,Chinese food,Guangdong,12,Chinese food,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
#引入商铺信息，整合进类别
SHOP_INFO_EN = pd.read_csv('./Flamingo_Rank4_code/data_new/SHOP_INFO_EN.csv')
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN, category, on = ['SHOP_CA1_EN','SHOP_CA2_EN','SHOP_CA3_EN'],how = 'left')
SHOP_INFO_EN['SHOP_CA1_EN'] = [(lambda x:(str(x.strip()) ) ) (x)  for x in SHOP_INFO_EN['SHOP_CA1_EN']] #去除空格

In [8]:
SHOP_INFO_EN.head()

,SHOP_ID,CITY,SHOP_LOC,SHOP_PAY,SHOP_SCO,SHOP_COM,SHOP_LEV,SHOP_CA1,SHOP_CA2,SHOP_CA3,...,SJ05,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点,...,0,1,0,0,0,0,0,0,0,0
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,0,1,0,0
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶,...,0,1,0,0,0,0,0,0,0,0
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN,...,0,0,0,0,0,0,0,1,0,0
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#引入用户支付信息
PAYNW = pd.read_csv('./Flamingo_Rank4_code/data_new/user_pay_new.csv')
PAYNW_TAB = pd.pivot_table(PAYNW, values=['Num_post'], index=['SHOP_ID'],columns=['DATE'], aggfunc=np.sum)   
PAYNW_TAB = pd.concat([PAYNW_TAB[PAYNW_TAB.columns[0:169:1]], pd.DataFrame({'A':[np.nan],},index=np.arange(1,2001)),PAYNW_TAB[PAYNW_TAB.columns[169::1]] ], axis = 1)
PAYNW_TAB.columns = [str((datetime.datetime.strptime('20150626','%Y%m%d') + datetime.timedelta(days=x)).date()) for x in range( PAYNW_TAB.shape[1])] 

In [10]:
PAYNW_TAB.head()

,2015-06-26,2015-06-27,2015-06-28,2015-06-29,2015-06-30,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,...,2016-10-22,2016-10-23,2016-10-24,2016-10-25,2016-10-26,2016-10-27,2016-10-28,2016-10-29,2016-10-30,2016-10-31
SHOP_ID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,202.0,243.0,215.584963,255.0,172.0,226.0,257.0,251.0,280.584963,180.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,155.0,143.0,78.000000,82.0,97.0,83.0,96.0,96.0,115.000000,80.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.0,125.0,47.000000,80.0,61.0,68.0,103.0,102.0,104.000000,67.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,211.0,221.0,81.000000,90.0,104.0,55.0,109.0,215.0,195.000000,115.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,161.0,240.0,453.584963,302.0,1.0,NaN,215.0,203.0,291.000000,239.0


In [50]:
#选择2015年有双11销量的商家
tt = PAYNW_TAB.loc[:,'2015-10-28':'2015-11-26'] 
tt = tt[tt.count(axis = 1)>=30] #
tt2 = PAYNW_TAB.loc[PAYNW_TAB.index.isin(tt.index), '2015-10-28':'2015-11-25']
tt3=tt2.reset_index() #重设索引  重要！！！

In [52]:
#双11销量增量，计算方法为上一年(2015年)11-11当天销量V1111与其前后两周对应工作日V1028，V1104，V1118，V1125的加权销量的比值,权重系数分别为$0.15,0.35,0.35,0.15$.
tt3['ratio'] = tt3['2015-11-11']/( 0.15*tt3['2015-10-28'] + 0.35*tt3['2015-11-04'] + 0.35*tt3['2015-11-18'] + 0.15*tt3['2015-11-25'])
tt3 = pd.merge(tt3,SHOP_INFO_EN, on = ['SHOP_ID'], how = 'left' )

In [53]:
tt3.head()

,SHOP_ID,2015-10-28,2015-10-29,2015-10-30,2015-10-31,2015-11-01,2015-11-02,2015-11-03,2015-11-04,2015-11-05,...,SJ05,SJ06,SJ07,SJ08,SJ09,SJ10,SJ11,SJ12,SJ13,SJ14
0,1,310.000000,212.000000,331.0,347.0,381.0,332.000000,403.584963,392.0,496.0,...,0,1,0,0,0,0,0,0,0,0
1,7,153.584963,102.584963,145.0,195.0,159.0,130.000000,165.000000,126.0,149.0,...,0,0,0,0,0,0,0,0,0,1
2,11,24.584963,14.000000,14.0,10.0,9.0,21.000000,22.000000,29.0,18.0,...,0,0,0,0,0,1,0,0,0,0
3,14,174.000000,151.000000,123.0,123.0,100.0,146.584963,135.000000,138.0,147.0,...,0,0,0,0,0,0,0,0,0,0
4,16,67.000000,78.000000,102.0,89.0,99.0,118.000000,97.000000,84.0,72.0,...,0,0,0,0,0,1,0,0,0,0


In [55]:
#商铺相关特征
SHOP_FEATURE = pd.read_csv("./SHOP_FEATURES.csv",low_memory=False)
SHOP_SC = ['SC00']
SHOP_SD = map(lambda x:'SD'+ str(x).zfill(2), np.arange(6)) 
SHOP_SE = map(lambda x:'SE'+ str(x).zfill(2), np.arange(1))
SHOP_SF = map(lambda x:'SF'+ str(x).zfill(2), np.arange(1))
SHOP_SG = map(lambda x:'SG'+ str(x).zfill(2), np.arange(4))
SHOP_SH = map(lambda x:'SH'+ str(x).zfill(2), np.arange(2))
SHOP_SI =  [(lambda x:('SI'+ str(x).zfill(2))) (x)  for x in range(10)]
SHOP_columns = SHOP_SC + SHOP_SD + SHOP_SE + SHOP_SF + SHOP_SG +  SHOP_SH + SHOP_SI

SHOP_FEATURE = SHOP_FEATURE[ ['SHOP_ID']  + SHOP_SC + SHOP_SE + SHOP_SF + SHOP_SG  + SHOP_SI]

In [56]:
tt3 = pd.merge(tt3[['SHOP_ID','SHOP_PAY','SHOP_SCO','SHOP_COM', 'SHOP_LEV','ratio']+SHOP_SJ], SHOP_FEATURE , on = ['SHOP_ID'],how = 'left')
SHOP_INFO_EN = pd.merge(SHOP_INFO_EN[['SHOP_ID','SHOP_PAY','SHOP_SCO','SHOP_COM', 'SHOP_LEV']+SHOP_SJ], SHOP_FEATURE , on = ['SHOP_ID'],how = 'left')

In [57]:
#通过三分之一左右已知的双11增长率，预测所有商家的双11增长率
import copy
SHOP_INFO_EN2 = copy.deepcopy(SHOP_INFO_EN)

del tt3['SHOP_ID']
Y  = tt3[['ratio']].values

del tt3['ratio']
X  = tt3.values

del SHOP_INFO_EN['SHOP_ID']
Xtest = SHOP_INFO_EN.values

In [58]:
#采用xgboost单模型训练，由于双11当天对应的工作日不同，2015年数据并不能很好反映出2016年双11节假日情况，且超市便利店类商店存在大量的数据缺失。为防止过拟合，参数设置均较为保守，最大深度为2，且加了较大的$L_1$正则项
XGBR = xgb.XGBRegressor(max_depth = 2,learning_rate=0.01,n_estimators=500,reg_alpha=10,gamma = 1)
XGBR.fit(X,Y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=10, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [59]:
ytest = XGBR.predict(Xtest)
print(ytest.mean())
print(ytest.min())
print(ytest.max())
print(ytest.std())
SHOP_INFO_EN2['DOU11'] = ytest*0.6+1.0*0.4

1.11167
0.733148
1.50733
0.202985


In [60]:
SHOP_INFO_EN2.to_csv('DOU11_coef.csv',index = False)